In [1]:
!pip install ultralytics easyocr opencv-python -qq kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.5/783.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.6 MB/s eta 0:00:00


In [2]:
import cv2
import re
import os
import numpy as np
import pandas as pd
import easyocr

from PIL import Image
from sklearn.metrics import classification_report, accuracy_score
from ultralytics import YOLO
from difflib import SequenceMatcher
from google.colab.patches import cv2_imshow
from google.colab import drive

In [40]:
drive.mount('drive', force_remount=True)

Mounted at drive


In [6]:
base_folder = 'drive/MyDrive/dataset-plat'

In [48]:
# data_train = pd.read_csv(f"{base_folder}/DataTrain.csv", sep=';')
data_val = pd.read_csv(f"{base_folder}/DataVal.csv", sep=';')
new_data_train = pd.read_csv(f"{base_folder}/new_data_train.csv")

In [43]:
new_data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Vehicleregistrationplate  1126 non-null   object
 1   NameofFile                1126 non-null   object
dtypes: object(2)
memory usage: 17.7+ KB


In [18]:
# haarcascadeplatenumber_folder = f"/content/drive/MyDrive/haarcascadeplatenumber/extracted_files/dataset"
# targeted_folder = f"{base_folder}/train"

# # Print the number of images in the source folder
# print(len(os.listdir(haarcascadeplatenumber_folder)))

# # Initialize the iteration count based on the existing data
# iteration = data_train.shape[0] + 1

# # Iterate over each image in the source folder
# new_entries = []

# # Iterate over each image in the source folder
# for img in os.listdir(haarcascadeplatenumber_folder):
#     if img.endswith(".jpg"):  # Ensure it's a JPG file
#         plate_number = img.split('.')[0]
#         new_img_name = f"DataTrain{iteration}.png"

#         # Convert the image from JPG to PNG and save it to the target folder
#         img_pil = Image.open(f"{haarcascadeplatenumber_folder}/{img}")
#         print(img_pil.filename)
#         img_pil.save(f"{targeted_folder}/{new_img_name}", "PNG")

#         # Append the new data entry to the list
#         new_entries.append([iteration - 1, plate_number, new_img_name])

#         # Increment the iteration count
#         iteration += 1

# # Create a DataFrame for the new entries
# new_data = pd.DataFrame(new_entries, columns=['Unnamed: 0', 'Vehicleregistrationplate', 'NameofFile'])

# # # Concatenate the new data to the existing DataFrame
# new_data_train = pd.concat([data_train, new_data], ignore_index=True)

In [38]:
# new_data_train.to_csv('new_data_train.csv', index=False)

In [20]:
plate_codes = {
    "A": "Banten, kecuali Kabupaten Tangerang (wilayah barat), Tangerang Selatan, dan Kota Tangerang",
    "AA": "Jawa Tengah eks Karesidenan Kedu: Kabupaten Magelang, Kota Magelang, Purworejo, Kebumen, Temanggung, Wonosobo",
    "AB": "Yogyakarta",
    "AD": "Jawa Tengah, eks Keraton Surakarta: Surakarta, Sukoharjo, Boyolali, Sragen, Karanganyar, Wonogiri, Klaten",
    "AE": "Jawa Timur, eks Karesidenan Madiun: Kabupaten Madiun, Kota Madiun, Ngawi, Magetan, Ponorogo, Pacitan",
    "AG": "Jawa Timur, eks Karesidenan Kediri: Kabupaten Kediri, Kota Kediri, Kabupaten Blitar, Kota Blitar, Tulungagung, Nganjuk, Trenggalek",
    "B": "Jakarta, Depok, Kabupaten Tangerang (wilayah timur), Kota Tangerang, Tangerang Selatan, Bekasi, Kabupaten Bekasi",
    "BA": "Sumatra Barat",
    "BB": "Sumatra Utara, wilayah pantai barat: Dairi, Pakpak Bharat, Samosir, Toba Samosir, Tapanuli Utara, Humbang Hasundutan, Tapanuli Tengah, Sibolga, Tapanuli Selatan, Padangsidempuan, Padang Lawas, Padang Lawas Utara, Mandailing Natal, Gunungsitoli, Nias, Nias Selatan, Nias Barat, Nias Utara",
    "BD": "Bengkulu",
    "BE": "Lampung",
    "BG": "Sumatra Selatan",
    "BH": "Jambi",
    "BK": "Sumatra Utara, wilayah pantai timur: Medan, Binjai, Deli Serdang, Langkat, Karo, Serdang Bedagai, Tebing Tinggi, Simalungun, Pematangsiantar, Batubara, Asahan, Tanjungbalai, Labuhan Batu, Labuhan Batu Utara, Labuhan Batu Selatan",
    "BL": "Aceh",
    "BM": "Riau",
    "BN": "Bangka Belitung",
    "BP": "Kepulauan Riau",
    "D": "Jawa Barat, eks Karesidenan Priangan Barat: Kabupaten Bandung, Kota Bandung, Cimahi, Bandung Barat",
    "DA": "Kalimantan Selatan",
    "DB": "Daratan Sulawesi Utara",
    "DC": "Sulawesi Barat",
    "DD": "Sulawesi Selatan, wilayah selatan: Makassar, Gowa, Maros, Pangkajene Kepulauan, Takalar, Jeneponto, Bulukumba, Bantaeng, Selayar",
    "DE": "Maluku",
    "DG": "Maluku Utara",
    "DH": "Nusa Tenggara Timur: Timor",
    "DK": "Bali",
    "DL": "Sulawesi Utara: Kepulauan Sangihe, Kepulauan Talaud, Kepulauan Sitaro",
    "DM": "Gorontalo",
    "DN": "Sulawesi Tengah",
    "DP": "Sulawesi Selatan, wilayah utara: Barru, Parepare, Pinrang, Sidenreng Rappang, Enrekang, Tana Toraja, Toraja Utara, Luwu, Palopo, Luwu Utara, Luwu Timur",
    "DR": "Nusa Tenggara Barat: Pulau Lombok",
    "DT": "Sulawesi Tenggara",
    "DW": "Sulawesi Selatan, wilayah tengah: Bone, Soppeng, Wajo, Sinjai",
    "E": "Jawa Barat, eks Karesidenan Cirebon: Kabupaten Cirebon, Kota Cirebon, Indramayu, Majalengka, Kuningan",
    "EA": "Nusa Tenggara Barat: Pulau Sumbawa",
    "EB": "Nusa Tenggara Timur: Pulau Flores, Alor, Lembata",
    "ED": "Nusa Tenggara Timur: Pulau Sumba",
    "F": "Jawa Barat, eks Karesidenan Bogor: Kabupaten Bogor, Kota Bogor, Cianjur, Kabupaten Sukabumi, Kota Sukabumi",
    "G": "Jawa Tengah, eks Karesidenan Pekalongan: Kabupaten Pekalongan, Kota Pekalongan, Kabupaten Tegal, Kota Tegal, Brebes, Batang, Pemalang",
    "H": "Jawa Tengah, eks Karesidenan Semarang: Kabupaten Semarang, Kota Semarang, Salatiga, Kendal, Demak",
    "K": "Jawa Tengah, eks Karesidenan Pati: Pati, Kudus, Jepara, Rembang, Blora, Grobogan",
    "KB": "Kalimantan Barat",
    "KH": "Kalimantan Tengah",
    "KT": "Kalimantan Timur",
    "KU": "Kalimantan Utara",
    "L": "Jawa Timur: Surabaya",
    "M": "Jawa Timur: Pulau Madura",
    "N": "Jawa Timur, eks Karesidenan Malang: Kabupaten Malang, Kota Malang, Kabupaten Pasuruan, Kota Pasuruan, Lumajang, Batu",
    "P": "Jawa Timur, eks Karesidenan Besuki: Bondowoso, Situbondo, Jember, Banyuwangi",
    "PA": "Papua, Papua Tengah, Papua Pegunungan",
    "PB": "Papua Barat",
    "PS": "Papua Selatan",
    "PY": "Papua Barat Daya",
    "R": "Jawa Tengah, eks Karesidenan Banyumas: Banyumas, Cilacap, Purbalingga, Banjarnegara",
    "S": "Jawa Timur, eks Karesidenan Bojonegoro: Bojonegoro, Kabupaten Mojokerto, Kota Mojokerto, Tuban, Lamongan, Jombang",
    "SB": "Surabaya: Becak",
    "T": "Jawa Barat, eks Karesidenan Karawang: Purwakarta, Karawang, Subang",
    "W": "Jawa Timur, eks Karesidenan Surabaya di luar Kota Surabaya: Sidoarjo, Gresik",
    "Z": "Jawa Barat, eks Karesidenan Priangan Timur: Garut, Kabupaten Tasikmalaya, Kota Tasikmalaya, Sumedang, Ciamis, Pangandaran, Banjar"
}

In [47]:
import os
from PIL import Image

def create_yolo_labels(df, images_path):
    labels_path = os.path.join(images_path, 'labels')
    os.makedirs(labels_path, exist_ok=True)
    for index, row in df.iterrows():
        image_name = row['NameofFile']
        label = row['Vehicleregistrationplate']

        # Define the path to the image
        image_path = os.path.join(images_path, image_name)

        # Load the image to get its dimensions
        with Image.open(image_path) as img:
            width, height = img.size

        # Placeholder bounding box (assume full image)
        # This is just a placeholder and should be replaced with actual bounding box coordinates
        x_center = 0.5
        y_center = 0.5
        bbox_width = 1.0
        bbox_height = 1.0

        # Write to YOLO format
        label_file_name = image_name.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(labels_path, label_file_name)
        with open(label_path, 'w') as f:
            f.write(f"0 {x_center} {y_center} {bbox_width} {bbox_height}\n")

# Paths to images
train_images_path = 'path/to/train'
test_images_path = 'path/to/test'

# Create labels
create_yolo_labels(train_df, train_images_path)
create_yolo_labels(test_df, test_images_path)

DataTest.csv  DataTrain.csv  new_data_train.csv  test  train


In [50]:
new_data_train

,Vehicleregistrationplate,NameofFile
0,A7814,DataTrain1.png
1,B1074QO,DataTrain2.png
2,B1031QO,DataTrain3.png
3,B187EDA,DataTrain4.png
4,B1089VD,DataTrain5.png
...,...,...
1121,S4073JB0,DataTrain1122.png
1122,S4136SI,DataTrain1123.png
1123,T6277CH,DataTrain1124.png
1124,W4324NCH,DataTrain1125.png


In [89]:
%cd /content/drive/MyDrive/plate-number-detection

/content/drive/MyDrive/plate-number-detection


In [83]:
!rm .gitignore
!touch .gitignore

In [95]:
## append drive and sample_data folder to gitignore add new line for every item ignored
with open('.gitignore', 'a') as f:
    f.write('\n')
    f.write('drive/')
    f.write('\n')
    f.write('sample_data/')
    f.write('\n')
    f.write('.config/')

In [98]:
!git remote add origin